# temporal - E03 - Analyzing multiyear patterns
### Revised on: 2025-10-28

This example focus on the analysis of monthly Wolf sunspot number. Data is available from 1749 (Source: [WDC-SILSO](\href{http://www.sidc.be/silso/datafiles), Royal Observatory of Belgium, Brussels). The signal contains the well-known 11 years Schwabe cycle, the 22 years one and smaller time scales variations described in [[1]](#1). The analysis focus on the detection of these cycles from 22 years down to the seasonal scale. The steps will be the following: 

1. Load environmentaltools packages
2. Read the input data and create the dictionary to fit the non-stationary probability model (PM)
3. Call environmentaltolls.temporal.analysis.marginalfit for fitting the parameters of the PM

## 1. Load environmentaltools packages

The following code load the basic functions (read data, analysis and plots) included in environmentaltools

In [ ]:
from environmentaltools.common import read
from environmentaltools.temporal import analysis
from environmentaltools.graphics import plots

## 2. Read the input data and create the dictionary to make the marginal fit

As sometimes happens, data did not come in datetime format. In this example the data is read adding a dateparser to create a pandas DataFrame which indexes are datetimes. The following code read the file of monthly sunspots. The input file is a txt file. As usually, some noise is included to ensure that the input variable is continuous and not discrete.

In [ ]:
from datetime import datetime
import pandas as pd

monthly_date_parser = lambda x: datetime.strptime(x, "%Y-%m")

data = pd.read_csv(
    "data/sunspots.txt",
    index_col=0,
    parse_dates=["Month"],
    date_parser=monthly_date_parser,
)
data = analysis.nanoise(data, "Sunspots")

Once the data is read, it is create the dictionary with the properties about the temporal expansion of parameters and the probability models. In this example, a piecewise function compound by two models was selected, where a lognorm PM will fit the body and a Gaussian model will fit the upper tail. The percentil matching point is 0.85. As we are looking for larger oscillations, a basis period of 22 years will be set-up. Both PMs will be expanded in time using the sinusoidal temporal expansion and 44 terms to include the seasonal behaviour. This information is translated to the dictionary as follow.

In [ ]:
params = {
    "var": "Sunspots",
    "fun": {0: "lognorm", 1: "norm"},
    "non_stat_analysis": True,
    "basis_functions": {
        "method": "modified",
        "noterms": 44,
    },
    "ws_ps": [0.85],
    "basis_period": [22],
    "piecewise": True,
}

## 3. Make the marginal fit
The following code will fit the parameters to the data.

In [ ]:
analysis.marginalfit(data, params)

The optimization of this case takes more time than example 01 due to the tool is optimizing 88 times the parameters (44 times per 2 probability models) and more than 400 parameteres are been optimized (3 parameters of the lognorm per 2 parameters of each expansion order per order 44 plus the parameters of the Gaussian probability model). The result will be saved to the file "Sunspots_lognorm_norm_0.85_nonst_22_modified_44_SLSQP.json" in the folder "marginalfit". Read carefully the information that the tool gives in the shell to understand that the parameters are passed correctly. You can also read the json file with any text editor and follows the results as in the previous dictionary.

## 4. Verificate the marginal fit

The non-stationary empirical and theoretical cumulative distribution functions are plotted. The following code shows some more options to modify the windows scale, that differs from the previous example while it is monthly timeseries. As it can be observed, some properties of the plot such as the windows lenght for the empirical non-stationary cumulative distribution function, the percentiles and the option to include an second real time axis can be chosen.

In [ ]:
import numpy as np

params = read.rjson("marginalfit/Qd_weibull_max_nonst_1_sinusoidal_10_SLSQP")

plots.nonstationary_cdf(
    data,
    "Sunspots",
    params["Sunspots"],
    date_axis=True,
    daysWindowsLength=30,
    equal_windows=True,
    pemp=np.array([0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]),
)

More information about this analysis can be found in [[2]](#2) and [[3]](#3).

## References
<a id="1">[1]</a> 
Usoskin, IG and Mursula, K (2003). 
Long-term solar cycle evolution: review of recent developments
Solar Physics, 218(1), 319-343.


<a id="2">[2]</a> 
M. Cobos, P. Otiñar, P. Magaña, A. Lira-Loarca, A. Baquerizo (2021). 
MarineTools.temporal (v 1.0.0): A Python package to simulate Earth and environmental timeseries
Submitted to Environmental Modelling & Software.


<a id="3">[3]</a> 
Cobos, M., Otíñar, P., Magaña, P., Baquerizo, A. (2021).
A method to characterize and simulate climate, earth or environmental vector random processes. 
Submitted to  Probabilistic Engineering & Mechanics.